In [107]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [108]:
print("Num GPUs Available: ",
      len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [109]:
TRAIN_DIR = './dataset/sdd/train/'
TEST_DIR = './dataset/sdd/test/'
IMG_SIZE = (150, 150)
BATCH_SIZE = 128
SEED = 1337

In [110]:
# Training generator
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_ds = train_datagen.flow_from_directory(TRAIN_DIR,
                                             target_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             seed=SEED,
                                             shuffle=True,
                                             class_mode='categorical')

# Test generator
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_ds = test_datagen.flow_from_directory(TEST_DIR,
                                           target_size=IMG_SIZE,
                                           batch_size=BATCH_SIZE,
                                           seed=SEED,
                                           shuffle=False,
                                           class_mode='categorical')

train_num = train_ds.samples
test_num = test_ds.samples

Found 16229 images belonging to 120 classes.
Found 4111 images belonging to 120 classes.


In [111]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(GlobalAveragePooling2D())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Activation('relu'))
model.add(Dense(120))
model.add(Activation('softmax'))

In [112]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_342 (Conv2D)          (None, 148, 148, 32)      896       
_________________________________________________________________
activation_415 (Activation)  (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_114 (MaxPoolin (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_343 (Conv2D)          (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_416 (Activation)  (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_115 (MaxPoolin (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_344 (Conv2D)          (None, 36, 36, 64)      

In [116]:
EPOCHS = 100

checkpoint_path = "checkpoint/dog/cp.ckpt"

model.load_weights(checkpoint_path)

cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              save_weights_only=True,
                              save_best_only=True,
                              mode='min',
                              verbose=1)

history = model.fit_generator(train_ds,
                              steps_per_epoch=train_num // BATCH_SIZE,
                              epochs=EPOCHS,
                              validation_data=test_ds,
                              validation_steps=test_num // BATCH_SIZE,
                              callbacks=[cp_callback])

Epoch 1/100
126/126 [==============================] - ETA: 0s - loss: 1.6970 - acc: 0.5248
Epoch 00001: val_loss improved from inf to 2.62318, saving model to checkpoint/dog\cp.ckpt
126/126 [==============================] - 94s 745ms/step - loss: 1.6970 - acc: 0.5248 - val_loss: 2.6232 - val_acc: 0.3806
Epoch 2/100
126/126 [==============================] - ETA: 0s - loss: 1.7236 - acc: 0.5236
Epoch 00002: val_loss did not improve from 2.62318
126/126 [==============================] - 94s 744ms/step - loss: 1.7236 - acc: 0.5236 - val_loss: 2.6785 - val_acc: 0.3701
Epoch 3/100
126/126 [==============================] - ETA: 0s - loss: 1.6731 - acc: 0.5307
Epoch 00003: val_loss did not improve from 2.62318
126/126 [==============================] - 94s 744ms/step - loss: 1.6731 - acc: 0.5307 - val_loss: 2.6748 - val_acc: 0.3887
Epoch 4/100
126/126 [==============================] - ETA: 0s - loss: 1.6717 - acc: 0.5304
Epoch 00004: val_loss did not improve from 2.62318
126/126 [=======

KeyboardInterrupt: 

In [117]:
model.save('boku_net.h5')

model = tf.keras.models.load_model('boku_net.h5')

model.evaluate(test_ds, batch_size=BATCH_SIZE)

33/33 [==============================] - 6s 183ms/step - loss: 2.7042 - acc: 0.3858


[2.7041656970977783, 0.3857942223548889]